# Зашумление и расшумление изображений с помощью сверточной нейронной сети (Keras)

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
import matplotlib.pyplot as plt

import os
import PIL.Image as Img

folder = 'train'
new_folder = 'cats'
px = 64
feat = px**2*3
for i in range(12500):
    name = '.'.join(['cat', str(i), 'jpg'])
    source = os.path.join(folder, name)
    dest = os.path.join(new_folder, name)
    img = Img.open(source)
    img = img.resize((px, px), Img.LANCZOS)
    img.save(dest)

source_input = (px, px, 3)
n_input = feat
n_hidden_1 = 2048
n_hidden_2 = 2048
n_output = n_input
training_epochs = 20
batch_size = 100
noise = 0.1

x = np.zeros((1100,) + source_input)
for i in range(1100):
    name = '.'.join(['cat', str(i), 'jpg'])
    dest = os.path.join(new_folder, name)
    x[i] = plt.imread(dest)

x_train, x_test = x[:1000], x[1000:]
#x_train = x_train.reshape((-1, feat))
#x_test = x_test.reshape((-1, feat))
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = x_train.copy()
y_test = x_test.copy()

x_train = x_train + noise * np.random.randn(*x_train.shape)
x_test = x_test + noise * np.random.randn(*x_test.shape)

plt.imshow(y_test[0].reshape(source_input))
plt.savefig("test_image.png")
plt.imshow(x_test[0].reshape(source_input)
plt.savefig("noise_image.png")

model = Sequential()
model.add(Conv2D(filters=96, kernel_size=3, padding="same", activation='relu',  input_shape=source_input))
model.add(Dropout(rate=0.01))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(filters=96, kernel_size=3, padding="same", activation="relu"))
model.add(Dropout(rate=0.01))
model.add(UpSampling2D())
model.add(Conv2D(filters=96, kernel_size=3, padding="same", activation="relu"))
model.add(Conv2D(filters=3, kernel_size=3, padding="same", activation="relu"))


model.compile(optimizer='adam', loss='mean_squared_error')

for i in range(training_epochs):
    print("Epoch", i+1)
    model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
    y_pred = model.predict(x_test, batch_size=batch_size)
    plt.imshow(y_pred[0].reshape(source_output))
    plt.savefig("denoised_image_"+str(i+1)+".png")
